In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import os
from PIL import Image
import rasterio
import time
from graph_tool.all import *

# Data information:

* rasterwgs84 is 976, 1760, main folder is ca 2500x3000

np.mgrid might be useful

In [ ]:
path_files = "../../data"

In [ ]:
def read_in_tifs(path):
    files = os.listdir(path)
    tif_list = []
    file_list = []
    for f in files:
        if f[-3:]=="tif":
            img = Image.open(os.path.join(path,f))
            tif_list.append(np.array(img))
            file_list.append(f[:-4])
    tif_arr = np.array(tif_list)
    tif_arr = tif_arr/255.
    print("shape of tif array:", tif_arr.shape)
    return tif_arr, file_list

In [ ]:
tifs, files = read_in_tifs(path_files)
plt.imshow(tifs[8,:,:])
plt.colorbar()
plt.show()

### Hard constraints

### Corridor

In [ ]:
def get_corridor(path, fn = "Corridor_BE.tif"):
    with rasterio.open(os.path.join(path,fn), 'r') as ds:
        arr = ds.read() 
    corr_img = Image.fromarray(arr[0])
    corr_resized = corr_img.resize((3022, 2627),resample=Image.BILINEAR)
    corridor = (np.array(corr_resized)<9900).astype(int)
    # plt.imshow(corridor)
    # plt.colorbar()
    # plt.show()
    return corridor

def normalize(instance):
    return (instance -
            np.min(instance)) / (np.max(instance) - np.min(instance))


In [ ]:
corridor = get_corridor(os.path.join(path_files, "corridor"))

In [ ]:
# get cost surface
with rasterio.open(os.path.join(path_files, "corridor/COSTSURFACE.tif"), 'r') as ds:
    arr = ds.read() 
print(arr.shape)
costs = normalize(arr[0])

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(costs)
plt.colorbar()
plt.show()

### Other hard constraints

In [ ]:
hard_cons_path = os.path.join(path_files, "hard_constraints")
hard_cons_arr, files = read_in_tifs(hard_cons_path)
# add corridor
hard_cons_arr = np.concatenate((hard_cons_arr, np.expand_dims(corridor, axis=0)), axis=0)
print(hard_cons_arr.shape)

In [ ]:
# logical and between all hard constraints
hard_constraints = np.all(hard_cons_arr.astype(int), axis=0)

In [ ]:
plt.imshow(hard_constraints)
plt.show()

### Sum tifs

In [ ]:
summed = np.sum(tifs, axis=0)

In [ ]:
summed.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(summed)
plt.colorbar()
plt.show()

### Define circle coordinates

In [ ]:
def get_donut(radius_low, radius_high):
    img_size = int(radius_high + 10)
    # xx and yy are 200x200 tables containing the x and y coordinates as values
    # mgrid is a mesh creation helper
    xx, yy = np.mgrid[-img_size:img_size, -img_size:img_size]
    # circle equation
    circle = (xx) ** 2 + (yy) ** 2
    # donuts contains 1's and 0's organized in a donut shape
    # you apply 2 thresholds on circle to define the shape
    donut = np.logical_and(circle < (radius_high**2), circle > (radius_low**2))
    pos_x, pos_y = np.where(donut>0)
    return pos_x-img_size, pos_y-img_size

def get_half_donut(radius_low, radius_high, vec):
    pos_x, pos_y = get_donut(radius_low, radius_high)
    new_tuples = []
    # vector = np.asarray(vec)
    # vector = vector/np.linalg.norm(vector)
    for i, j in zip(pos_x, pos_y):
        if i*vec[0] + j*vec[1] >=0:
        # point = np.asarray([i,j]).T
        # if np.dot(point)>0:
        # if i>0 or (i==0 and j>0):
            new_tuples.append((i,j))
    return new_tuples

In [ ]:
# example:
upper = 5.5
img_size = int(upper)+1
pos_x, pos_y = get_donut(2.5,upper)

# new donut
new_tuples =  get_half_donut(2.5,upper, (-1,1))
print(new_tuples)

# whole donut
ar = np.zeros((2*img_size,2*img_size))
for i in range(len(pos_x)):
    ar[pos_x[i]+img_size, pos_y[i]+img_size]=1
plt.imshow(ar)
plt.show()

# modified donut
ar = np.zeros((2*img_size,2*img_size))
for tup in new_tuples:
    ar[tup[0]+img_size, tup[1]+img_size]=1
plt.imshow(ar)
plt.show()
print(len(new_tuples))

### Scale down instance

In [ ]:
def reduce_instance(summed, square):
    x_len,y_len = summed.shape
    new_img = np.zeros((x_len//square, y_len//square))
    for i in range(x_len//square):
        for j in range(y_len//square):
            patch = summed[i*square:(i+1)*square, j*square:(j+1)*square]
            new_img[i,j] = np.mean(patch)
    return new_img

In [ ]:
instance = reduce_instance(summed, 16)
instance_norm = (instance-np.min(instance))/(np.max(instance)-np.min(instance))

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(instance_norm)
# plt.colorbar()
plt.show()

In [ ]:
instance_corr = reduce_instance(hard_constraints, 16)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(instance_corr)
# plt.colorbar()
plt.show()

In [ ]:
instance_norm.shape

## Networkx graph

In [ ]:
def pos2node(pos, length):
    return pos[0]*length + pos[1]
def node2pos(node, length):
    j = node%length # rest
    i = node//length
    return i,j

### Define nodes

In [ ]:
pos_x, pos_y = get_donut(2.5,5)

In [ ]:
# node_list = ["{},{}".format(str(i), str(i)) for i in range(10)] # with strings
x_len, y_len = instance_norm.shape
print(x_len, y_len)
node_list = [(pos2node((i,j), y_len),{"pos":(i,j)}) for i in range(x_len) for j in range(y_len) if instance_corr[i,j]]
# assert len(np.unique([n[0] for n in node_list]))==x_len*y_len

In [ ]:
node_list_wo_attr = [n[0] for n in node_list]

### Build edge list

In [ ]:
import time

In [ ]:
tic = time.time()
inds_x, inds_y = np.where(instance_corr>0)
donut_tuples = get_half_donut(2.5,5)
edge_list = []

for i, j in zip(inds_x, inds_y):
    weight_node = 1-instance_norm[i,j]
    node_name = pos2node((i,j), y_len)
    for (x,y) in donut_tuples:
        new_x = i+x
        new_y = j+y
        if new_x>=0 and new_x<x_len and new_y>=0 and new_y<y_len and instance_corr[new_x,new_y]: # inside corridor
            weight = 1-instance_norm[new_x, new_y]+weight_node
            edge_list.append((node_name, pos2node((new_x,new_y), y_len), {"weight": round(weight,1)}))
            
print(time.time()-tic)

In [ ]:
print(len(edge_list))

### Add edges and nodes to graph

In [ ]:
g = nx.Graph()
g.add_nodes_from(node_list)
g.add_edges_from(edge_list)

### Plot the graph with edge attributes etc

In [ ]:
def plot_graph(g):
    labels = nx.get_edge_attributes(g,'weight') # returns dictionary
    pos = nx.get_node_attributes(g,'pos')
    plt.figure(figsize=(20,10))
    nx.draw(g,pos)
    nx.draw_networkx_edge_labels(g, pos, edge_labels=labels)
    # plt.savefig("first_graph.png")
    plt.show()
plot_graph(g)

### Run shortest path algorithm

In [ ]:
first_node = 9 # node_list[0][0]
last_node = 277 # node_list[-1][0]
path = nx.bellman_ford_path(g,first_node, last_node)

### Plot path

In [ ]:
path_positions = [node2pos(v, y_len) for v in path]

### Path plotting function

In [ ]:
def plot_path(instance, path):
    # expand to greyscale
    expanded = np.expand_dims(instance, axis=2)
    expanded = np.tile(expanded, (1,1,3)) # overwrite instance by tiled one
    # colour nodes in path in red
    for (x,y) in path:
        expanded[x,y] = [0.9, 0.2, 0.2]
        
    plt.figure(figsize=(20,10))
    plt.imshow(expanded)
    plt.show()

## Graph Tools version

### Define node to pos and pos to node mappings

In [ ]:
# new version: node - pos list and pos -> node dictionary
x_len, y_len = instance_norm.shape

node_pos = [(i,j) for i in range(x_len) for j in range(y_len) if instance_corr[i,j]]
pos_node = {node_pos[i]:i for i in range(len(node_pos))}

pos2node = np.ones(instance_norm.shape)
pos2node *= -1
for n, (i,j) in enumerate(node_pos):
    pos2node[i,j] = n

### Define graph and add nodes

In [ ]:
tic = time.time()

G = Graph(directed=False)
weight = G.new_edge_property("float")

# add nodes to graph
vlist = G.add_vertex(len(node_pos))
print("added nodes:", len(list(vlist)))

print("time to add nodes to graph", time.time()-tic)

### Add edges: new version

In [ ]:
def shift_surface_general(costs, shift):
    if shift[0]<0:
        tup1 = (0,-shift[0])
    else:
        tup1 = (shift[0],0)
    if shift[1]<0:
        tup2 = (0,-shift[1])
    else:
        tup2 = (shift[1],0)
        
    costs_shifted = np.pad(costs, (tup1,tup2), mode='constant')
    
    if shift[0]>0 and shift[1]>0:
        costs_shifted = costs_shifted[:-shift[0], :-shift[1]]
    elif shift[0]>0 and shift[1]<=0:
        costs_shifted = costs_shifted[:-shift[0], -shift[1]:]
    elif shift[0]<=0 and shift[1]>0:
        costs_shifted = costs_shifted[-shift[0]:, :-shift[1]]
    elif shift[0]<=0 and shift[1]<=0:
        costs_shifted = costs_shifted[-shift[0]:, -shift[1]:]
        
    return costs_shifted

In [ ]:
def shift_surface(costs, shift):
    rolled_costs = np.roll(costs, shift, axis=(0,1))
    rolled_costs[:shift[0], :] = 0
    if shift[1] >= 0:
        rolled_costs[:, :shift[1]] = 0
    else:
        rolled_costs[:, shift[1]:] = 0
    return rolled_costs

In [ ]:
arr = np.linspace(0,1,81).reshape((9,9))

In [ ]:
shift_surface(arr, (1,-4))

In [ ]:
shift_surface_general(arr, (1,-4))

### For testing --> delete

In [ ]:
def bresenham_line(x0, y0, x1, y1):
    """
    find pixels on line between two pixels
    https://stackoverflow.com/questions/50995499/generating-pixel-values-of-line-connecting-2-points

    """
    steep = abs(y1 - y0) > abs(x1 - x0)
    if steep:
        x0, y0 = y0, x0
        x1, y1 = y1, x1

    switched = False
    if x0 > x1:
        switched = True
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    if y0 < y1:
        ystep = 1
    else:
        ystep = -1

    deltax = x1 - x0
    deltay = abs(y1 - y0)
    error = -deltax / 2
    y = y0

    line = []
    for x in range(x0, x1 + 1):
        if steep:
            line.append([y, x])
        else:
            line.append([x, y])

        error = error + deltay
        if error > 0:
            y = y + ystep
            error = error - deltax
    if switched:
        line.reverse()
    return line


def get_kernel(shifts):
    """
    Get all kernels describing the path of the edges in a discrete raster
    :param shifts: possible circle points
    :returns kernel: all possible kernels (number of circle points x upper x upper)
    :returns posneg: a list indicating whether it is a path to the left (=1) or to the right(=0)
    """
    upper = np.amax(np.absolute(shifts)) + 1
    posneg = []
    kernel = np.zeros((len(shifts), upper, upper))

    for i, shift in enumerate(shifts):
        if shift[1] < 0:
            posneg.append(1)
            line = bresenham_line(0, upper - 1, shift[0], upper - 1 + shift[1])
        else:
            posneg.append(0)
            line = bresenham_line(0, 0, shift[0], shift[1])
        # add points of line to the kernel
        for (j, k) in line:
            kernel[i, j, k] += 1
    return kernel, posneg


def convolve(img, kernel, neg=0):
    k_size = len(kernel)
    if neg:
        padded = np.pad(img, ((0, k_size - 1), (k_size - 1, 0)))
    else:
        padded = np.pad(img, ((0, k_size), (0, k_size)))
    # print(padded.shape)
    convolved = np.zeros(img.shape)
    w, h = img.shape
    for i in range(0, w):
        for j in range(0, h):
            patch = padded[i:i + k_size, j:j + k_size]
            convolved[i, j] = np.sum(patch * kernel)
    return convolved

def shift_surface(costs, shift):
    rolled_costs = np.roll(costs, shift, axis=(0, 1))
    rolled_costs[:shift[0], :] = 0
    if shift[1] >= 0:
        rolled_costs[:, :shift[1]] = 0
    else:
        rolled_costs[:, shift[1]:] = 0
    return rolled_costs

tic = time.time()

G = Graph(directed=False)
weight = G.new_edge_property("float")

# add nodes to graph
vlist = G.add_vertex(len(node_pos))
print("added nodes:", len(list(vlist)))

print("time to add nodes to graph", time.time()-tic)

In [ ]:
# umdefinitions of variables
costs = 1-instance_norm
corrgreater0 = (instance_corr>0).astype(int)
costs_rest = costs*corrgreater0
# plt.imshow(costs*corrgreater0)
# plt.show()
shifts = donut_tuples

orig_greater_zero = costs_rest>0
inds_orig = pos2node[costs_rest>0]

kernels, posneg = get_kernel(shifts)

for i in range(len(shifts)):
    # print(shifts[i], shift_tuples[i])
    costs_shifted = shift_surface(costs_rest, shifts[i])
    
    # both_greater_zero = np.all(np.asarray([orig_greater_zero, costs_shifted>0]), axis=0)
    # weights = (costs_shifted + costs_rest)/2
    weights = convolve(costs_rest, kernels[i], posneg[i])
    
    inds_shifted = pos2node[costs_shifted>0]
    # delete the ones where inds_shifted is zero
    assert len(inds_shifted)==len(inds_orig)
    weights_list = weights[costs_shifted>0]
    
    pos_inds = inds_shifted>=0
    out = np.swapaxes(np.asarray([inds_orig, inds_shifted, weights_list]), 1,0)[pos_inds]
    # print(out.shape)
    # print(out[:100])
    G.add_edge_list(out, eprops=[weight])

### Add edges: old version

In [ ]:
tic = time.time()
# inds_x, inds_y = np.where(instance_corr>0)
donut_tuples = get_half_donut(2.5,5)
edge_list = []

for n, (i, j) in enumerate(node_pos):
    # n is the name of the node in the graph (=index), (i,j) the position
    weight_node = 1-instance_norm[i,j]
    for (x,y) in donut_tuples:
        new_x = i+x
        new_y = j+y
        if new_x>=0 and new_x<x_len and new_y>=0 and new_y<y_len and instance_corr[new_x,new_y]: # inside corridor
            weight = 1-instance_norm[new_x, new_y]+weight_node
            edge_list.append([n, pos_node[(new_x,new_y)], round(weight,3)])
            
print("time to build edge list", time.time()-tic)

In [ ]:
tic = time.time()

# add edges and properties to the graph
G.add_edge_list(edge_list, eprops=[weight])
print("added edges:", len(list(G.edges())))

print("time to add edges and nodes to graph", time.time()-tic)

### Add start and end vertex

In [ ]:
def add_start_end_vertices(G, instance_corr, pos2node, start_list=None, end_list=None):
    # defaults if no start and end list are given:
    topbottom, leftright = np.where(instance_corr) # change 
    if start_list is None:
        nr_start = len(topbottom)//100
        start_list = zip(topbottom[:nr_start], leftright[:nr_start])
    if end_list is None:
        nr_end = len(topbottom)//100
        end_list = zip(topbottom[-nr_end:], leftright[-nr_end:])
        
    neighbor_lists = [start_list, end_list]
    start_and_end = []

    for k in [0,1]:
        v = G.add_vertex()
        v_index = G.vertex_index[v]
        start_and_end.append(v)
        print("index of start/end vertex", v_index)
        edges = []
        for (i,j) in neighbor_lists[k]:
            neighbor_ind = pos2node[i,j]
            edges.append([v_index, neighbor_ind, 0])
        G.add_edge_list(edges, eprops=[weight])
    
    return start_and_end

add_start_end_vertices(G, instance_corr, pos2node, start_list=None, end_list=None)

### Compute shortest path

In [ ]:
tic = (time.time())
source = 7075
target = 7076
vertices_path, edges_path = shortest_path(G, G.vertex(source), G.vertex(target), weights=weight, negative_weights=True) # true for bellman ford
gt_path = [node_pos[G.vertex_index[v]] for v in vertices_path[1:-1]]
print("time for shortest path", time.time()-tic)

### Track why edges costs path take so many turns

In [ ]:
for e in G.vertex(vertices_path[1]).out_edges():
    print(e)
    print(G.ep.weight[e])
    
for e in edges_path:
    print(G.ep.weight[e])

### Plot path

In [ ]:
def plot_path(instance, path):
    # expand to greyscale
    expanded = np.expand_dims(instance, axis=2)
    expanded = np.tile(expanded, (1,1,3)) # overwrite instance by tiled one
    # colour nodes in path in red
    for (x,y) in path:
        
        expanded[x-2:x+2,y-2:y+2] = [0.9, 0.2, 0.2]
        
    plt.figure(figsize=(25,15))
    plt.imshow(expanded)
    plt.show()

In [ ]:
plot_path(instance_norm, gt_path)

In [ ]:
print(gt_path)

In [ ]:
print(gt_path)

# Graph IO

In [ ]:
# need to have internal property map https://graph-tool.skewed.de/static/doc/quickstart.html#internal-property-maps
G.edge_properties["weight"] = weight

In [ ]:
G.save("my_graph.xml.gz")
print(weight[G.edge(66, 69)]) # error because weight also used as variable in edge_list definition

In [ ]:
# G2 = load_graph("my_graph.xml.gz")
weight = G2.ep.weight[G2.edge(66, 69)]
print(weight)

In [ ]:
weights_mapping = G2.ep.weight

# Old stuff

In [ ]:
def get_shift_transformed(shifts):
    
    shift_tuples = []
    for shift in shifts:
        if shift[0]<0:
            tup1 = (0,-shift[0])
        else:
            tup1 = (shift[0],0)
        if shift[1]<0:
            tup2 = (0,-shift[1])
        else:
            tup2 = (shift[1],0)
        shift_tuples.append((tup1,tup2))
    
    return shift_tuples

donut_tuples = get_half_donut(2.5,5)
shift_tuples = get_shift_transformed(donut_tuples)

In [ ]:
# old version to add edges to gt graph
for edge in edge_list:
    e = G.add_edge(edge[0], edge[1])
    weight[e] = edge[2]["weight"]


In [ ]:
# Old version to form edge list
edge_list = []
for i in range(x_len):
    for j in range(y_len):
        node_name = pos2node((i,j), y_len)
        if node_name in node_list_wo_attr:
            weight_node = 1-instance_norm[i,j]
            for x,y in zip(pos_x, pos_y):
                new_x = i+x
                new_y = j+y
                if new_x>=0 and new_x<x_len and new_y>=0 and new_y<y_len and instance_corr[new_x,new_y]: # inside corridor
                    weight = 1-instance_norm[new_x, new_y]+weight_node
                    edge_list.append((node_name, pos2node((new_x,new_y), y_len), {"weight": round(weight,1)}))
                    # TODO: INVERT edge weight
                    

                    
# FROM MAIN PY FILE:

tic = time.time()
# inds_x, inds_y = np.where(instance_corr>0)
donut_tuples = get_half_donut(2.5, 5)
edge_list = []
for n, (i, j) in enumerate(node_pos):
    # n is the name of the node in the graph (=index), (i,j) the position
    weight_node = 1 - instance_norm[i, j]
    for (x, y) in donut_tuples:
        new_x = i + x
        new_y = j + y
        # if inside the image at all
        if new_x >= 0 and new_x < x_len and new_y >= 0 and new_y < y_len:
            # if inside corridor
            if instance_corr[new_x, new_y]:
                weight = 1 - instance_norm[new_x, new_y] + weight_node
                edge_list.append(
                    (
                        n, pos_node[(new_x, new_y)], {
                            "weight": round(weight, 3)
                        }
                    )
                )
print("time to build edge list:", time.time() - tic)

# construct graph
tic = (time.time())
G = Graph(directed=False)
weight = G.new_edge_property("float")
vlist = G.add_vertex(len(node_pos))  # nodes: indices of node_list
print("added nodes:", len(list(vlist)))
for edge in edge_list:
    e = G.add_edge(edge[0], edge[1])
    weight[e] = edge[2]["weight"]
print("time to build up the graph:", time.time() - tic)

# get shortest path
tic = (time.time())
SOURCE = 0
TARGET = len(node_pos) - 1
vertices_path, edges_path = shortest_path(
    G,
    G.vertex(SOURCE),
    G.vertex(TARGET),
    weights=weight,
    negative_weights=True
)  # true for bellman ford
path = [node_pos[G.vertex_index[v]] for v in vertices_path]
print("time for shortest path", time.time() - tic)

# new version: node - pos list and pos -> node dictionary

In [ ]:
l = [(1,2), (3.4, 5.5)]
import json
with open("test.json", "w") as outfile:
    json.dump(l, outfile)

In [ ]:
with open("test.json", "r") as outfile:
    l_new = json.load(outfile)

In [ ]:
l_new